In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import gc
import os
from pathlib import Path
#from argparse import Namespace
#from collections import defaultdict
import matplotlib as plt

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
#from scipy import stats

import warnings
warnings.filterwarnings("ignore")


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Reduce Memory Size

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                #if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #    df[col] = df[col].astype(np.float16)
                #elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                #    df[col] = df[col].astype(np.float32)
                #else:
                df[col] = df[col].astype(np.float16)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

Import data

In [ ]:
#train = pd.read_csv('../input/ubiquant-market-prediction/train.csv')
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
train =train[0:1000000]

In [ ]:
import gc
gc.collect()
test_cols = ['time_id','investment_id']
feature = [c for c in train.columns if "f_" in c]
features = ['investment_id','time_id'] + feature

len(feature)

In [ ]:
train['time_id']=pd.to_numeric(train['time_id'])
train['time_id']=train['time_id'].astype(np.float16)
train['investment_id']=pd.to_numeric(train['investment_id'])
train['investment_id']=train['investment_id'].astype(np.float16)
train['investment_id2']=train['investment_id']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(train[test_cols])
train[test_cols]=scaler.transform(train[test_cols])
train[test_cols]=train[test_cols].astype(np.float16)

In [ ]:
_=train.pop('row_id')
investment_id2=train.pop('investment_id2')
y=train.pop('target')

In [ ]:
gc.collect()

In [ ]:
import tensorflow as tf
def preprocess(X, y):
    return X, y
def make_dataset(feature, y, batch_size=1024, mode="train"):
    ds = tf.data.Dataset.from_tensor_slices(((feature), y))
    

    ds = ds.map(preprocess)
    if mode == "train":
        ds = ds.shuffle(4096)
    
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
gc.collect()


In [ ]:
def get_model():
    #time_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    #investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((302, ), dtype=tf.float16)
    
    feature_x = layers.Dense(512, activation='swish')(features_inputs)
    feature_x = layers.Dense(512, activation='swish')(feature_x)
    feature_x = layers.Dense(512, activation='swish')(feature_x)
    
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(feature_x)
    #x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    #x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    #x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x) 
    #x = layers.Dense(64, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x) 
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.0002), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model

In [ ]:
model = get_model()
model.summary()
keras.utils.plot_model(model, show_shapes=True)


In [ ]:
len(features)

In [ ]:
%%time
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(2, shuffle=True, random_state=42)
models = []
gc.collect()
for index, (train_indices, valid_indices) in enumerate(kfold.split(train[features], investment_id2)):
    
    X_train, X_val = train.iloc[train_indices], train.iloc[valid_indices]
    #time_id_train = time_id[train_indices]
    #investment_id_train = time_id[train_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[valid_indices]
    #investment_id_val = time_id[valid_indices]
    #time_id_val = time_id[valid_indices]
    train_ds = make_dataset(X_train, y_train)
    valid_ds = make_dataset(X_val, y_val, mode="valid")
    model = get_model()
    checkpoint = keras.callbacks.ModelCheckpoint(f"model_{index}", save_best_only=True)
    early_stop = keras.callbacks.EarlyStopping(patience=3)
    model.fit(train_ds, epochs=50, validation_data=valid_ds, callbacks=[checkpoint, early_stop])
    model = keras.models.load_model(f"model_{index}")
    models.append(model)
    #pearson_score = stats.pearsonr(model.predict(valid_ds).ravel(), y_val.values)[0]
    #print('Pearson:', pearson_score)
    #pd.DataFrame(history.history, columns=["mse", "val_mse"]).plot()
    #plt.title("MSE")
    #plt.show()
    #pd.DataFrame(history.history, columns=["mae", "val_mae"]).plot()
    #plt.title("MAE")
    #plt.show()
    #pd.DataFrame(history.history, columns=["rmse", "val_rmse"]).plot()
    #plt.title("RMSE")
    #plt.show()

    del X_train
    del X_val
    del y_train
    del y_val
    del train_ds
    del valid_ds
    gc.collect()
    break
    

In [ ]:
def preprocess_test(feature):
    return (feature), 0
def make_test_dataset(feature, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices(((feature)))
    ds = ds.map(preprocess_test)
    #ds = ds.batch(batch_size)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds
def inference(models, ds):
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append(y_pred)
    return np.mean(y_preds, axis=0)

In [ ]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test() 
for (test_df, sample_prediction_df) in iter_test:
    
    test_df['time_id']=test_df['row_id'].apply(lambda x:x[0:x.find('_')])
    test_df['time_id']=pd.to_numeric(test_df['time_id'], downcast='float')
    test_df['time_id']=test_df['time_id'].astype(np.float16)
    test_df['investment_id']=pd.to_numeric(test_df['investment_id'], downcast='float')
    test_df['investment_id']=test_df['investment_id'].astype(np.float16)
    test_df[test_cols]=scaler.transform(test_df[test_cols])
    ds = make_test_dataset(test_df[features])
    #print(test_df[features])
    sample_prediction_df['target'] = inference(models, ds)
    #print(sample_prediction_df)
    env.predict(sample_prediction_df) 